# Projeto 1 - Ciência dos Dados

Nome: Vinícius de Barros Gomes

Nome: Gabriel Nakano Fujihara

Nome: Pedro Roberto Fernandes Noronha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [16]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [17]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Insper\2023\1-semestre\DSoft(desing de software)\DataScienceProject


Carregando a base de dados com os tweets classificados manualmente:

In [18]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,"Target (Loja e Sistema, Entrega,Conteudo)",Unnamed: 2,"Target (neutro, positivo, negativo)"
0,"Não li, até pq não consegui baixar. Vcs podem ...",Loja e Sistema,NaN,neutro
1,Não tem nada indicando que é em português de P...,Loja e Sistema,NaN,neutro
2,"Produto horrível, veio com várias páginas em b...",Entrega,NaN,neutro
3,O minimo é não ler esse livro,Conteúdo do Livro,NaN,negativo
4,Parece que o autor não leu o próprio livro!! 🙄,Conteúdo do Livro,NaN,negativo


In [19]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,"Target (Neutro, Positivo, Negativo)"
0,"Apesar da narrativa prender o leitor, os argum...",negativo
1,A formatação desse ebook atrapalha muito a lei...,neutro
2,Não é o primeiro livro da Record que oxida em ...,neutro
3,O hype que envolve esse livro é ENORME então e...,negativo
4,"meu deus, essa escritora conseguiu estragar um...",negativo


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Buscamos avaliar se os livros oferecidos pela amazon são ou não bons, portanto, foi ignorado quando haviam críticas exclusivamente ao serviço da amazon, como entrega e ou qualidade física do produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

language = "portuguese"
stopwordsportugues=stopwords.words(language)
#As funções filtrador e limpar caracteres servem para iltrar o df deixando tudo minúsculo e eliminando caracteres especiais e stopwords
def filtrador(df):
    comentarios = df["Mensagem"]
    data_frame =pd.DataFrame({"Mensagem":[]})
    target=pd.DataFrame({"Target (Loja e Sistema, Entrega,Conteudo)":df["Target (Loja e Sistema, Entrega,Conteudo)"]})
    for i in comentarios:
        frase = i.lower()
        frase=limpar_caracteres(frase)
        frase=' '.join([k for k in frase.split(" ") if  k not in stopwordsportugues])
        data_frame.loc[len(data_frame)]={"Mensagem":frase}
    x=data_frame["Mensagem"]
    y=df["Target (Loja e Sistema, Entrega,Conteudo)"]
    final_df=pd.DataFrame({"Mensagem":x,"Target (Loja e Sistema, Entrega,Conteudo)":y})
    return final_df
train = filtrador(train)
def limpar_caracteres(x):
    x = re.sub("@[A-Za-z0-9_]+","", x)

    x = re.sub("#[A-Za-z0-9_]+","", x)

    x = re.sub(r'http\S+', '', x)

    x = re.sub('[()!?ㅋ;:,/^~♂️*=&-]', ' ', x)

    x = re.sub('\[.*?\]',' ', x)

    x = re.sub("[z0-9]"," ", x)

    x = re.sub('\\\\\\\\n'," ", x)

    x = re.sub('\\\\'," ", x)

    x = re.sub('\ '," ", x)
    
    x = re.sub("é","e",x)
    
    x = re.sub("ã","a",x)
    return re.sub(r'[^\w\s]','',x)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo']

In [14]:
#Retorna quantidade de palavras totais em um data frame
def quantidade_da_palavra(df):
    todas_as_frases=df["Mensagem"]
    todas_as_palavras = []
    for i in todas_as_frases:
        x=i.split(" ")
        for p in x:
            todas_as_palavras.append(p)
    todas_as_palavrasSeries=pd.Series(todas_as_palavras)
    Vl=todas_as_palavrasSeries.value_counts()
    return Vl
quantidade_da_palavra_Entrega=quantidade_da_palavra(df_Entrega)
quantidade_da_palavra_Loja_Sistema=quantidade_da_palavra(df_Loja_Sistema)
quantidade_da_palavra_Conteudo=quantidade_da_palavra(df_Conteudo)

C:\Users\flama\AppData\Local\Temp\ipykernel_9564\2261098339.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  todas_as_palavrasSeries=pd.Series(todas_as_palavras)


In [15]:
def classificador(df):
    palavra=str(input("Escreva a palavra ou frase a ser testada"))
    palavra=palavra.lower()
    target=str(input("Escreva o target"))
    df=df.loc[df["Target (Loja e Sistema, Entrega,Conteudo)"]==target]
    print(df)
    if " " in palavra:
        palavra=list(palavra.split(" "))
    palavras={}
    for i in palavra:
        palavras[i]=0
    probabilidade=1
    total=0
    aparicoes_da_palavra=0
    for i in df["Mensagem"]:
        frase= list(i.split(" "))
        for e in frase:
            total+=1
            for u in palavras.keys():
                if e ==u:
                    palavras[u]+=1
    for i in palavras.values():
        probabilidade*=i/total
    print(total)
    return probabilidade
def retornatotal(df):
    df=filtrador(df)
    total=0
    df=df["Mensagem"]
    for i in df:
        e=list(i.split(" "))
        total+=len(e)
    print(total)
    return total
retornatotal(train)

def vl(lista):
    lista=filtrador(lista)
    todas_as_frases=lista["Mensagem"]
    todas_as_palavras = []
    for i in todas_as_frases:
        x=i.split(" ")
        for p in x:
            todas_as_palavras.append(p)
    todas_as_palavrasSeries=pd.Series(todas_as_palavras)
    Vl=todas_as_palavrasSeries.value_counts()
    return Vl

print(vl(train))

26455
              4206
nao            858
livro          760
livros         157
autor          120
              ... 
decaído          1
cá               1
gaste            1
centaur          1
considerar       1
Length: 5882, dtype: int64


In [19]:
def laplace(w):
    palavra=str(input('palavra:'))
    lista_de_numeros=w
    target=str(input('target:'))
    if target=='Loja e Sistema':
        target_n=3303
    elif target=='Conteúdo do Livro':
        target_n=13350
    else:
        target_n=3628   
    laplace_r=(lista_de_numeros[palavra]+1)/(target_n+21143)
    return laplace_r
print(laplace(vl(train)))

0.002045324388448008


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**